In [37]:
#LIBRARIES
import pandas as pd
import numpy as np
import quandl
from dotenv import load_dotenv
import os

In [84]:
# GETS THE API KEY FROM .ENV FILE
def get_key(d_key):
    load_dotenv('.env')
    key = os.environ.get(d_key)
    return key

# GETS DATA FROM QUANDL / DEFAULT IS BOING DATA 
def get_data(code="EOD/BA", start_date="1970-01-01", end_date="2019-05-27"):
    return quandl.get(code, start_date=start_date, end_date=end_date)

# KEEP SIGNIFICANT COLUMNS. ADD CHANGE AND VOLATILITY
def feature_eng(data):
    data = data[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]
    data.loc['Volatility'] = (data['Adj. High'] - data['Adj. Low']) / data['Adj. Low']
    data.loc['Change'] = (data['Adj. Close'] - data['Adj. Open']) / data['Adj. Open']
    return data

In [78]:
quandl_key = get_key('quandl_key')
quandl.ApiConfig.api_key = quandl_key
data = get_data('WIKI/GOOGL')

In [85]:
data_ready = feature_eng(data)
data_ready.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2004-08-19 00:00:00,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20 00:00:00,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23 00:00:00,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24 00:00:00,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25 00:00:00,52.542193,54.167209,52.100830,53.164113,9188600.0
